## DS4DS Final Task:

Upload your solution (training pipeline and final model ready for inference) as one archive file (.zip) to moodle at least three days prior to your exam appointments. Request an appointment at least two weeks in advance via email (oliver.wallscheid@uni-siegen.de). The latest exam date will be by end of September 2025. 

In [ ]:
using Weave
weave("final_task.ipynb", out_path="errorfree1.html") 

In [1]:
using Pkg
Pkg.activate(".")
Pkg.status()

  Activating project at `d:\ds4dsproject\final_task_data_and_aux_files\final_task`


Status `D:\ds4dsproject\final_task_data_and_aux_files\final_task\Project.toml` (empty project)


In [ ]:
# this might take a few minutes, but you should only need to do this once
# when you use the project for the first time
Pkg.precompile()

In [2]:
using MAT
using Plots 
using LaTeXStrings
using Serialization
using StatsBase
using Flux

include("utils.jl");

### Thermal modelling of an electric vehicle motor

Your task is to create a model for the thermal dynamics of an electric motor.
More formally, you need to find a parameterized model of the form

$$
\begin{align}
    \frac{\mathrm{d}}{\mathrm{d}t} \mathbf{x}(t) &= \mathcal{M}_\mathbf{w} (\mathbf{x}(t), \mathbf{u}(t)) \\
    \mathbf{y}(t) & = \mathbf{x} (t)
\end{align}
$$

where $\mathbf{x}(t)$ the state of the system and $\mathbf{u(t)}$ is the input vector. The state and
input signals are described further into this notebook. There are no boundaries on the specific model class and topology regarding $\mathcal{M}_\mathbf{w}$, that is, you are allowed to investigate freely which modelling approaches are suitable.

You will be given a dataset which you can use to train/fit your model. After submission, we will evaluate your model
on a different portion of the dataset to see its generalization performance.
This evaluation will be done by comparing state trajectories predicted by your model $\hat{\mathbf{X}}_i$ with the
true trajectory $\mathbf{X}_i$ using the mean squared error (MSE). The estimate $\hat{\mathbf{X}}_i$ is to be calculated 
based on an initial true state $\mathbf{x}_{0,i}$ (which is the first element of $\mathbf{X}_i$) and the true inputs 
$\mathbf{U}_i$ (or an interpolation of the true inputs). This means that your task is to build a simulation model that produces the state trajectory based on the
initial state and inputs.

**Considered grading criteria:**

- functional and tidy codebase
- successful inference on test set (no errors)
- model and feature innovations/concepts (beyond bare-bone black-box models)
- final presentation of proposed solution and used modeling rationale
- model accuracy (MSE) on unseen/hidden test data (not part of this folder)

### Load data:

In [3]:
data = matread("train_data.mat")

Dict{String, Any} with 6 entries:
  "XNames"             => Any["TpStatTc" "TpRotTc"]
  "U"                  => [500.01 0.38556 … 59.46 61.18; 500.02 0.392385 … 59.4…
  "X"                  => [65.402 67.427; 65.3795 67.4218; … ; 81.6888 86.2227;…
  "measurement_series" => [1; 1; … ; 10; 10;;]
  "dt"                 => 0.5
  "UNames"             => Any["NSft" "TqSftClcdInv" … "TpOilRotRetA" "TpOilRotR…

In [4]:
# extract data from dict

U = data["U"];
X = data["X"];
Y = X;

UNames = data["UNames"];
XNames = data["XNames"];
YNames = XNames;

measurement_series = data["measurement_series"];
dt = data["dt"];

### Data description:

You are given a part of a dataset measured from an electric motor at Mercedes-Benz to examine its thermal properties. The dataset you are given consists of $594885$ measurement points, which are divided into $10$ independent series of measurement sessions. A short description of the different variables in the data is given in the following:

---

- ```data["U"]``` contains a matrix with the shape $594885 \times 21$. These are the $594885$ samples of the input vector $\mathbf{u}(t) \in \mathbb{R}^{21}$ which can influence its thermal behavior. In detail these are
    - $u_1$: rpm, rotations per minute of the motor shaft
    - $u_2$: torque of the drive shaft
    - $u_3$: absolute value of the torque of the drive shaft
    - $u_4$: $i_d$ d-current
    - $u_5$: $i_q$ q-current
    - $u_6$: $\| i_q \|$
    - $u_7$: root-mean-squared phase current
    - $u_8$: $u_d$ d-voltage
    - $u_{9}$: $\| u_d \|$ 
    - $u_{10}$: $u_q$ q-voltage
    - $u_{11}$: neutral point displacement voltage
    - $u_{12}$: inverter switching frequency
    - $u_{13}$: DC link voltage
    - $u_{14}$: modulation index
    - $u_{15}$: power of the drive shaft
    - $u_{16}$: oil flow rate for cooling of the rotor
    - $u_{17}$: oil flow rate for cooling of the stator
    - $u_{18}$: oil temperature at entry (rotor)
    - $u_{19}$: oil temperature at entry (stator)
    - $u_{20}$: oil temperature at exit A (rotor)
    - $u_{21}$: oil temperature at exit B (rotor)
- ```data["UNames"]``` contains abbreviations for the different elements in $\mathbf{u}(t)$, e.g. for plot labels
- ```data["X"]``` contains the state of the motor, which in this case is a stator and a rotor temperature. There are $594885$ samples of the state vector (target) $\mathbf{x}(t) \in \mathbb{R}^{2}$.
    - $x_1$: $T_{stat}$ stator temperature
    - $x_2$: $T_{rot}$ rotor temperature
- ```data["XNames"]``` contains abbreviations for the different elements in $\mathbf{x}(t)$, e.g. for plot labels
- ```data["dt"]``` contains the time between measurement points
- ```data["measurement_series"]``` contains a matrix with the shape $594885 \times 1$. That assigns each measurement point to a measurement series, i.e. the $242856$-th element in this matrix is a $4$ which means that the $242856$-th measurement belongs to measurement series $4$.

**Hint 1:** What implication does the split into multiple measurement series have? Does it make sense to use all of the data at once?

**Hint 2:** While you might not have access to the actual test data, is there a way to improve/test the generalization performance of your model?

**Hint 3:** Through the  ```utils.jl```-file in the task folder you are given a set of minimal helper functions which you can use to get a feeling for the data and create some plots.

In [ ]:
?plot_at_idx

In [ ]:
p1 = plot(xlabel=L"k", ylabel=L"u");
p1 = plot_at_idx(p1, U, measurement_series, UNames, measurement_series_idx=4, feature_idx=21) 

In [ ]:
p2 = plot(xlabel=L"k", ylabel=L"y");
p2 = plot_at_idx(p2, Y, measurement_series, XNames, measurement_series_idx=8, feature_idx=2)

In [ ]:
p = plot_histogram(U, UNames, feature_idx=10)

### Create and fit your model:

In [ ]:
# REPLACE WITH YOUR CODE
parameters = [2, 2, 5, 1]-

if @isdefined parameters
    serialize("final_task_parameters", parameters)
end
# REPLACE WITH YOUR CODE

In [21]:
#functions for feature engg and data prep
function feature_engg(u)
    return u
end

function data_prep(U_raw, X_raw, measurement_series, series_indices, max_steps = nothing)
    inputs = []
    targets = []
    for series_idx in series_indices
        u_series = extract(U_raw, measurement_series, series_idx)
        x_series = extract(X_raw, measurement_series, series_idx)

        u_series = Float32.(u_series)
        x_series = Float32.(x_series)

        engineered_u = feature_engg(u_series)

        input_data = permutedims(engineered_u, (2, 1))
        target_data = permutedims(x_series, (2, 1))

        if max_steps !== nothing
            input_data = input_data[:, 1:max_steps]
            target_data = target_data[:, 1:max_steps]
        end

        push!(inputs, input_data)
        push!(targets, target_data)
    end
    return inputs, targets
end

data_prep (generic function with 2 methods)

In [35]:
#dividing the data
training_series_indices = 1
val_series_indices = 6
test_series_indices = 9:10
steps = 10

# 1. First, get all data
training_inputs, training_targets = data_prep(U, X, measurement_series, training_series_indices, steps)
val_inputs, val_targets = data_prep(U, X, measurement_series, val_series_indices, steps)
test_inputs, test_targets = data_prep(U, X, measurement_series, test_series_indices, steps)

(Any[Float32[499.98 500.01 … 500.06 499.96; 0.38744998 0.38839498 … 0.36970502 0.384195; … ; 74.67 74.67 … 74.57 74.57; 75.76 75.76 … 75.93 75.89], Float32[499.92 500.1 … 499.99 499.91; 0.38913 0.39753 … 0.355635 0.40194002; … ; 55.24 55.19 … 54.87 54.84; 55.08 55.05 … 54.73 54.71]], Any[Float32[77.68367 77.67987 … 77.66153 77.65891; 83.07999 83.08416 … 83.085884 83.08202], Float32[69.90183 69.90491 … 69.929955 69.93315; 85.22991 85.18944 … 84.90354 84.8472]])

In [38]:
using Statistics

all_training_inputs = hcat(training_inputs...)
μ_input = mean(all_training_inputs, dims=2)
σ_input = std(all_training_inputs, dims=2)

all_training_targets = hcat(training_targets...)
μ_target = mean(all_training_targets, dims=2)
σ_target = std(all_training_targets, dims=2)

normalize(data, μ, σ) = (data .- μ) ./ σ
denormalize(data, μ, σ) = (data .* σ) .+ μ

normalized_training_inputs = [normalize(x, μ_input, σ_input) for x in training_inputs]
normalized_training_targets = [normalize(y, μ_target, σ_target) for y in training_targets]
normalized_val_inputs = [normalize(x, μ_input, σ_input) for x in val_inputs]
normalized_val_targets = [normalize(y, μ_target, σ_target) for y in val_targets]

1-element Vector{Matrix{Float32}}:
 [-6.7888007 -7.107344 … -9.264978 -9.553661; -27.777664 -27.85833 … -28.471687 -28.675396]

In [40]:
#new normalisation
using Statistics

all_training_inputs = hcat(training_inputs...)
min_input = minimum(all_training_inputs)
max_input = maximum(all_training_inputs)

all_training_targets = hcat(training_targets...)
min_target = minimum(all_training_targets)
max_target = maximum(all_training_targets)

normalize(data, min_val, max_val) = (data .- min_val) ./ (max_val .- min_val)
denormalize(data, min_val, max_val) = (data .* (max_val .- min_val)) .+ min_val

normalized_training_inputs = [normalize(x, min_input, max_input) for x in training_inputs]
normalized_training_targets = [normalize(y, min_target, max_target) for y in training_targets]
normalized_val_inputs = [normalize(x, min_input, max_input) for x in val_inputs]
normalized_val_targets = [normalize(y, min_target, max_target) for y in val_targets]

1-element Vector{Matrix{Float32}}:
 [-0.16422176 -0.17410736 … -0.24106672 -0.25002566; 0.73210436 0.73136234 … 0.7257203 0.72384644]

In [18]:
unique(size(x, 1) for x in training_inputs)
println("Training data shapes:")
for i in 1:length(training_inputs)
    println("Sample $i: input size = ", size(training_inputs[i]), ", target size = ", size(training_targets[i]))
end


Training data shapes:
Sample 1: input size = (21, 20), target size = (2, 20)


In [ ]:
sample_input = training_inputs[1]
sample_target = training_targets[1]

println("Sample input size: ", size(sample_input))
println("Sample target size: ", size(sample_target))

Flux.reset!(model)
output = model(sample_input[:, 1:2])  # Forward pass on first 2 timesteps

println("Model output size: ", size(output))


Sample input size: (21, 30)
Sample target size: (2, 30)


UndefVarError: UndefVarError: `model` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [ ]:
using Flux, Flux.Optimise
using Flux: params

#model
num_input_features = size(training_inputs[1], 1)
num_output_features = size(training_targets[1], 1)

hidden_size = 64
model_lstm = Chain(
    LSTM(num_input_features => hidden_size), 
    Dense(hidden_size, num_output_features)
)

evaluate_model (generic function with 1 method)

In [41]:
# Check the min and max values of your training inputs
min_val = minimum(minimum.(normalized_training_inputs))
max_val = maximum(maximum.(normalized_training_inputs))
println("Min value: ", min_val)
println("Max value: ", max_val)

Min value: 0.0
Max value: 1.0


In [45]:
# Model (Part 1: The convolutional and pooling layers)
cnn_layers = Chain(
    # Correctly reshape and permute the input
    x -> PermutedDimsArray(reshape(x, size(x, 1), size(x, 2), 1), (2, 1, 3)),
    Conv((3,), num_input_features => 32, relu),
    MaxPool((2,)),
    Flux.flatten
)

# Get the size of the flattened output
sample_input = normalized_training_inputs[1]
flattened_size = size(cnn_layers(sample_input), 1)

# Model (Part 2: The full model)
model_cnn = Chain(
    cnn_layers,
    # The Dense layer is now dynamic
    Dense(flattened_size, num_output_features)
)

Chain(
  Chain(
    var"#83#84"(),
    Conv((3,), 21 => 32, relu),         # 2_048 parameters
    MaxPool((2,)),
    Flux.flatten,
  ),
  Dense(128 => 2),                      # 258 parameters
)                   # Total: 4 arrays, 2_306 parameters, 9.289 KiB.

In [ ]:
#hyperparamters
#model
model = model_cnn

#loss
mse(predicted, target) = mean(abs2, predicted .- target)

#optimizer
learning_rate = 0.00001
optimizer = Flux.setup(ADAM(learning_rate), model)

#epochs
epochs = 10

#eval
function evaluate_model(model, inputs, targets)
    total_loss = 0.0
    for (input, target) in zip(inputs, targets)
        Flux.reset!(model)
        predicted_normalized = model(input)
        predicted_denormalized = denormalize(predicted_normalized, min_target, max_target)
        total_loss += mse(predicted_denormalized, denormalize(target, min_target, max_target))

    end
    return total_loss / length(inputs)
end

evaluate_model (generic function with 1 method)

In [ ]:
# Corrected training loop
function train_model!(model, optimizer, opt_state, epochs, training_inputs, training_targets, val_inputs, val_targets)
    for epoch in 1:epochs
        for (inputs, targets) in zip(training_inputs, training_targets)
            # Reset the model's state at the start of each new sequence
            Flux.reset!(model)

            # Use `withgradient` for a more efficient way to get loss and gradients
            loss, grads = Flux.withgradient(model) do m
                predicted_traj = m(inputs)
                mse(predicted_traj, targets)
            end

            # Update the model and the optimizer state with the new gradients
            Flux.update!(opt_state, model, grads[1])
        end

        train_loss = evaluate_model(model, training_inputs, training_targets)
        validation_loss = evaluate_model(model, val_inputs, val_targets)
        
        println("Epoch $epoch, Training Loss: $train_loss, Validation Loss: $validation_loss")
    end
end

train_model!(model, optimizer, optimizer, epochs, normalized_training_inputs, normalized_training_targets, normalized_val_inputs, normalized_val_targets)

Epoch 1, Training Loss: NaN, Validation Loss: NaN
Epoch 2, Training Loss: NaN, Validation Loss: NaN
Epoch 3, Training Loss: NaN, Validation Loss: NaN
Epoch 4, Training Loss: NaN, Validation Loss: NaN
Epoch 5, Training Loss: NaN, Validation Loss: NaN
Epoch 6, Training Loss: NaN, Validation Loss: NaN
Epoch 7, Training Loss: NaN, Validation Loss: NaN
Epoch 8, Training Loss: NaN, Validation Loss: NaN
Epoch 9, Training Loss: NaN, Validation Loss: NaN
Epoch 10, Training Loss: NaN, Validation Loss: NaN


In [15]:
final_model = model

test_loss = evaluate_model(final_model, test_inputs, test_targets)
println("Final test Loss: $test_loss")

serialize("final_task_parameters", params(final_model))

Final test Loss: 2.440201875e6


┌ Warning: `Flux.params(m...)` is deprecated. Use `Flux.trainable(model)` for parameter collection,
│ and the explicit `gradient(m -> loss(m, x, y), model)` for gradient computation.
└ @ Flux C:\Users\Admin\.julia\packages\Flux\BkG8S\src\deprecations.jl:93


### Saving your model for evaluation:

After you have finished your model, we will evaluate its performance on the test dataset. To do so we will need to be able to make a forward pass with your model and we will need you to give us your model parameters.

- fill out the function given in ```model_forward_pass.jl```
- store your parameters in a file named ```final_task_parameters```

```
# Example code for storing parameters:

if @isdefined parameters
    serialize("final_task_parameters", parameters)
end
```

You can use the code below on the training to check if your result can be read properly **(If you run into problems with this or if your forward pass needs extra inputs, please contact us)**:

In [ ]:
include("model_forward_pass.jl");
using .ModelForwardPass

In [ ]:
?ModelForwardPass.your_model_forward_pass

In [ ]:
parameters_evaluation = deserialize("final_task_parameters");

measurement_series_idx = 1

_u = extract(U, measurement_series, measurement_series_idx);
_target = extract(X, measurement_series, measurement_series_idx);
_x0 = _target[1, :];

tsteps = collect(0:0.5:(size(_u, 1) * 0.5));

predicted_trajectory = ModelForwardPass.your_model_forward_pass(
    inputs=_u,
    x0=_x0,
    parameters=parameters_evaluation,
    tsteps=tsteps
);

println("MSE: ", StatsBase.mean((_target - predicted_trajectory).^2))

p_eval = plot(xlabel=L"k", ylabel=L"x")

p_eval = plot_at_idx(p_eval, X, measurement_series, XNames, measurement_series_idx=measurement_series_idx, feature_idx=1)
p_eval = plot!(p_eval, predicted_trajectory[:, 1], label=L"\hat{x}_1")

display(p_eval)

p_eval = plot(xlabel=L"k", ylabel=L"x")

p_eval = plot_at_idx(p_eval, X, measurement_series, XNames, measurement_series_idx=measurement_series_idx, feature_idx=2)
p_eval = plot!(p_eval, predicted_trajectory[:, 2], label=L"\hat{x}_2")

display(p_eval)